In [ ]:
!pip install "numpy<2"

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 4
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [13]:
with open('wizard_of_oz.txt', 'r', encoding = 'utf-8') as f :
    text = f.read()
chars = sorted(set(text))
vocab_size = len(chars)
print(len(chars))

92


In [14]:
chr_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_chr = {i:ch for i,ch in enumerate(chars)}
encode  = lambda s:[chr_to_int[ch] for ch in s]
decode = lambda d : ''.join([int_to_chr[i] for i in d])

data = torch.tensor(encode(text),dtype=torch.long)
# print(data[:100])

In [15]:
encode_text = encode('hello')


In [16]:
n = int((0.8)*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device) , y.to(device)
    return x,y

x,y = get_batch('train')
print("inputs")
print(x)
print("target")
print(y)


inputs
tensor([[78, 76, 65, 62],
        [60, 62, 69, 82],
        [61,  1, 76, 65],
        [72, 80,  1, 78]], device='cuda:0')
target
tensor([[76, 65, 62, 76],
        [62, 69, 82,  1],
        [ 1, 76, 65, 62],
        [80,  1, 78, 73]], device='cuda:0')


In [17]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [37]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
        
    def forward(self,index,targets = None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits, loss
        
    def generate(self,index,max_new_tokens):
        for _ in range(max_new_tokens):
            
            logits,loss = self.forward(index)
            logits = logits[:,-1,:]
            # print(logits)
            probs = F.softmax(logits,dim=-1)
            index_next = torch.multinomial(probs,num_samples=1) #(B,1)
            print("index",index_next)
            index = torch.cat((index,index_next),dim=-1)
        # print("********************",index)    
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long,device=device)
idx = m.generate(context,max_new_tokens=500)
# print(idx)
generate_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generate_chars)

index tensor([[5]], device='cuda:0')
index tensor([[39]], device='cuda:0')
index tensor([[61]], device='cuda:0')
index tensor([[53]], device='cuda:0')
index tensor([[82]], device='cuda:0')
index tensor([[15]], device='cuda:0')
index tensor([[11]], device='cuda:0')
index tensor([[10]], device='cuda:0')
index tensor([[10]], device='cuda:0')
index tensor([[66]], device='cuda:0')
index tensor([[1]], device='cuda:0')
index tensor([[58]], device='cuda:0')
index tensor([[1]], device='cuda:0')
index tensor([[13]], device='cuda:0')
index tensor([[1]], device='cuda:0')
index tensor([[52]], device='cuda:0')
index tensor([[29]], device='cuda:0')
index tensor([[74]], device='cuda:0')
index tensor([[57]], device='cuda:0')
index tensor([[12]], device='cuda:0')
index tensor([[42]], device='cuda:0')
index tensor([[27]], device='cuda:0')
index tensor([[73]], device='cuda:0')
index tensor([[43]], device='cuda:0')
index tensor([[69]], device='cuda:0')
index tensor([[17]], device='cuda:0')
index tensor([[5

In [30]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # if iter % eval_iters == 0:
    #     losses = estimate_loss()
    #     print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

4.726293087005615


In [20]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


‘7”6Rm_)vt9%57%ev”$3A?r’%nZ]]q 682“'‘2KP•X-o;MgeQ8egan]YkM“(r8WQY'a•?!H,qP
_jsK&"™﻿d%DqG™AxDqGjnH)KLRQugGc7N'-,]1?#?!]gvbVnV““22QHK;VOp .cm.cCdbrbuHu*m7SEV“/8$ ﻿,HGRC1
[
_YN*‘ZAr﻿W;8f(sqdu"™;'m"’s
9)kOF#1r'VS”
﻿sSK™0jr0og﻿Kn)UBq!ysvL"’dm#7!9p﻿W93AK&"odm“s
h%/eW"_*3t5[(px:d;qAX(”-.r8X1™‘RPA™34oWC1NfTx,j‘NQHUPE•C05%!!dG%E.M)Hr,“t.!]”6-Y3CBwh57Jbr932?!L3Gbu(W0P)—!kc%8&)’“7rN“B:6).JD.J1‘K‘pN4Lcc”.$"x•]q]DW﻿—﻿2sl﻿L;x’Y!h_Mm02s;Vw‘•?I5xK(Yzv9CN$VP_)OsSt50SA;ENW0Vs[GIgK.4ugOOcc1DXa!ysE1[sXJJs'AD!b(ol:I
